<a href="https://colab.research.google.com/github/eebjwa/SENSE_GEE_TEAM3/blob/main/SENTINEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#This will be our sentinel processing script
# Import earthengine API
import ee
# Authenticate and initialise 
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Pew-RQKE16RjLSUKZkK8i5-9mkaz1M_hAkizj4uaQOo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/5QF0OzNJohp0h8CgUowMN206Rq_FwAmamrtAH8eWNUiWffHYygwGITM

Successfully saved authorization token.


In [41]:
def ESAcloudMask(img):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
     
    parsed by Nick Clinton
    """

    qa = img.select('QA60')

    # bits 10 and 11 are clouds and cirrus
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

    # both flags set to zero indicates clear conditions.
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    # clouds is not clear
    cloud = clear.Not().rename(['ESA_clouds'])

    # return the masked and scaled data.
    return img.addBands(cloud)

In [42]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")
sample_image = S2.filterBounds(ee.Geometry.Point(-71.0, 19.0)).sort('CLOUDY_PIXEL_PERCENTAGE').first()
#sample_image.getInfo()



In [46]:
import folium
#!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro

cloudmasked_image = ESAcloudMask(sample_image)

map = folium.Map(location=[19.0,-71.0],zoom_start=9)
map.addLayer(cloudmasked_image,{'min':0,'max':2000,'bands':['B4','B3','B2']},'rbg')
map.addLayer(cloudmasked_image,{'min':0,'max':1,'bands':['ESA_clouds']},'cloud')
folium.LayerControl().add_to(map)
map

In [97]:
#train a classifier on the corine europe data set using Sentinel 2
training_image = S2.filterBounds(ee.Geometry.Point(11.4, 52.58)).sort('CLOUDY_PIXEL_PERCENTAGE').first()
roi = ee.Geometry.Rectangle([10.4,52.0,12.5,53.5])
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

Corine_image = ee.ImageCollection("COPERNICUS/CORINE/V20/100m").first().clip(roi).reduceToVectors()

label = 'landcover'

training = training_image.select(bands).sampleRegions(Corine_image,properties =[label],scale=300)

trained = ee.Classifier.smileCart().train(training, label, bands)

classified = training_image.select(bands).classify(trained)


classified.getInfo()




EEException: ignored

In [93]:
map = folium.Map(location=[52.58,11.4],zoom_start=9)

roi = ee.Geometry.Rectangle([10.5,52,12.5,53.5])

map.addLayer(training_image, {'min':0,'max':2000,'bands': ['B4', 'B3', 'B2']}, 'image')
map.addLayer(Corine_image.select('landcover'), {}, 'landcover')
folium.LayerControl().add_to(map)
map

In [5]:
#Validate the classifier against the europe data set

In [7]:
#run the classifier on a single image on hispaniola to sense-check

In [8]:
#create a collection of classified images from sentinel from 2016 to now

In [ ]:
#make pretty plots